## Exercise 0

In [3]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups

df_news_grp_u = []
df_news_grp = fetch_20newsgroups(categories = ['comp.graphics', 'sci.med'], subset = 'all', shuffle = True, random_state = 3116)

for i in df_news_grp.data:
    out = ""
    for j in i:
        if j.isalpha() or j.isnumeric() or j == " " or j == "\n":
            out = out + j 
        else:
            out = out + " "
    tmp = out.split()
    tmp2 = ""
    for s in tmp:
        if s not in set(stopwords.words('english')):
            tmp2 = tmp2 + s + " "
    df_news_grp_u.append(tmp2)

In [21]:
w_dict = []
corps = ""
for i in df_news_grp_u:
    corps = corps + i + " "
w_dict = np.array(list(set(corps.split())))

matX = np.zeros((len(df_news_grp_u), len(w_dict)))
for k, v in enumerate(df_news_grp_u):
    tmp = list(v.split())
    val, cnts = np.unique(tmp, return_counts = True) 
    for t in val:
        matX[k, w_dict == t] = cnts[val == t]

In [22]:
matTF = np.zeros(matX.shape)
for i in range(matTF.shape[0]):
    matTF[i, :] = matX[i, :] / np.sum(matX[i, :])

matIDF = np.zeros(matX.shape)
for i in range(matIDF.shape[1]):
    matIDF[:, i] = np.log(len(matX[:, i]) / np.count_nonzero(matX[:, i]))

mat_Tf_Idf = matTF * matIDF

In [23]:
def ttv_split(matX, matY, div):    
    train_x, train_y = matX[:math.ceil(div[0] * rows), :], matY[:math.ceil(div[0] * rows)]
    val_x, val_y = matX[math.ceil(div[0] * rows):math.ceil(div[1] * rows), :], matY[math.ceil(div[0] * rows):math.ceil(div[1] * rows)]
    test_x, test_y = matX[math.ceil(div[1] * rows):, :], matY[math.ceil(div[1] * rows):]
    
    return train_x, train_y, val_x, val_y, test_x, test_y

myList = [matX, mat_Tf_Idf]   
rows, cols = matX.shape
div = [0.8, 0.9]
c_name = ['BOW Approach', 'TFIDF Approach']
splt = {}

for valX, val in enumerate(myList):    
    train_x, train_y, val_x, val_y, test_x, test_y = ttv_split(val, df_news_grp.target, div)
    splt[c_name[valX]] = [train_x, train_y, val_x, val_y, test_x, test_y]


## Exercise 1

In [24]:

def prediction(v, i_prob, n_class, l_prob):
    p_prob = []
    for u, v in enumerate(n_class):
        pr = np.log(i_prob[u])
        p_prob.append(np.sum(np.log(l_prob[u, :]) * v) + pr) 
    return n_class[np.argmax(p_prob)] 

for key, val in splt.items():
    train_x, train_y, val_x, val_y, test_x, test_y = val
    r, c = train_x.shape
    n_class = np.unique(train_y) 
    num_class = len(n_class) 
    print(key)

    i_prob = np.zeros(num_class)
    l_prob = np.zeros((num_class, c))

    for u, v in enumerate(n_class):
        x_cls = train_x[v == train_y]
        i_prob[u] = x_cls.shape[0] / r 
        l_prob[u, :] = (x_cls.sum(axis = 0) + 1) / (np.sum(x_cls.sum(axis = 0) + 1))
    pred = np.zeros(val_x.shape[0])

    for u, v in enumerate(val_x):
        pred[u] = prediction(v, i_prob, n_class, l_prob)
    val_pred = pred

    val_a = np.sum(val_pred == val_y) / len(val_y)
    print('Val accuracy : ', val_a * 100)

    pred2 = np.zeros(test_x.shape[0])
    for u, v in enumerate(test_x):
        pred2[u] = prediction(v, i_prob, n_class, l_prob)

    test_pred = pred2

    test_a = np.sum(test_pred == test_y) / len(test_y)
    print('Test accuracy : ', test_a * 100)
    


BOW Approach
Val accuracy :  43.87755102040816
Test accuracy :  54.59183673469388
TFIDF Approach
Val accuracy :  43.87755102040816
Test accuracy :  54.59183673469388


## Exercise 2

In [28]:

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

h_param = {
    "C": [1., 1.2, 1.4],
    "kernel": ['poly', 'rbf', 'linear', 'sigmoid'],     
    "degree": [1, 2, 3],
    "gamma": ['scale']
}

for valX, val in enumerate(myList):
    if valX == 0:
        print('BOW Approach')
    else:
        print('TFIDF Approach')
        
    train_x, train_y, test_x, test_y, val_x, val_y = ttv_split(val, df_news_grp.target, [0.9, 1])
    svm = SVC(tol = 0.01, max_iter = 100)
    
    print("RSCV 3-fold")
    rscv = RandomizedSearchCV(svm, h_param, cv = 3, random_state = 3116)
    rscv_m = rscv.fit(train_x, train_y)

    opt_p = rscv_m.best_params_
    print('Optimal params: ', opt_p)

    test_pred = rscv_m.predict(test_x)
    test_accu = np.sum(test_pred == test_y) / len(test_y)
    print('Test accuracy: ', test_accu * 100)


BOW Approach
RSCV 3-fold
Optimal params:  {'kernel': 'linear', 'gamma': 'scale', 'degree': 2, 'C': 1.0}
Test accuracy:  87.24489795918367
TFIDF Approach
RSCV 3-fold
Optimal params:  {'kernel': 'rbf', 'gamma': 'scale', 'degree': 1, 'C': 1.4}
Test accuracy:  94.89795918367348
Wall time: 6min 42s
